### Import Library

In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from collections import Counter
from scipy.stats import  ks_2samp, chi2_contingency
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Dense, Dropout

import nest_asyncio
nest_asyncio.apply()

# تنظیم بذر تصادفی برای تکرارپذیری
# np.random.seed(42)
# tf.random.set_seed(42)

# تنظیم زمینه اجرایی محلی
tff.backends.native.set_local_execution_context()

C:\Users\TOP\anaconda3\envs\evn8\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.18.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Create Model 

In [2]:
# تابع ساخت مدل MLP
def create_mnist_mlp_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))  # ورودی به شکل مسطح شده (28x28 = 784)
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # لایه خروجی با 10 کلاس (softmax)
    optimizer = Adam(learning_rate=0.01)
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل

    return model

### Load and Split

In [6]:
# لود کردن داده‌های MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# نرمال‌سازی داده‌ها
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# مسطح کردن تصاویر
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# کدگذاری لیبل‌ها
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# تقسیم داده‌ها بین 10 کلاینت
num_clients = 30
client_data = np.array_split(x_train, num_clients)
client_labels = np.array_split(y_train, num_clients)

### Bug Injection

In [7]:
# لیستی از کلاینت‌هایی که باید دوبار نرمال‌سازی شوند
clients_to_double_normalize = [0,15,19]  # کلاینت‌های 3,6 (با اندیس صفر شروع می‌شود)

# دوبار نرمال‌سازی کلاینت‌های مشخص‌شده
for client_index in clients_to_double_normalize:
    client_data[client_index] = client_data[client_index] / 255.0

# پوشه‌ای برای ذخیره پارامترهای کلاینت‌ها ایجاد می‌کنیم
if not os.path.exists('client_params'):
    os.makedirs('client_params')

### Models training & save

In [8]:
# آموزش مدل‌ها برای هر کلاینت و ذخیره پارامترها
client_models = [] #لیست برای ذخیره مدل‌های هر کلاینت

base_model = create_mnist_mlp_model()  # مدل پایه

#base_model.save_weights('base_model_weights.h5') # ذخیره وزن‌های مدل پایه

for i in range(num_clients):
    print(f"Training model for client {i+1}")    
    model=create_mnist_mlp_model()
    model.set_weights(base_model.get_weights())
    
    # بررسی یکسان بودن وزن‌ها قبل از آموزش
    # m1 = model.get_weights()
    # b1 = base_model.get_weights()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # کامپایل کردن مدل
    # for index, (base_w, clone_w) in enumerate(zip(m1, b1)):
    #     if not np.array_equal(base_w, clone_w):
    #         print(f"Weights at index {index} are different.")
    #     else:
    #         print(f"Weights at index {index} are identical.")
    model.fit(client_data[i], client_labels[i], epochs=5, batch_size=32, verbose=1)
    #model.save_weights(f'client_params/client_{i+1}_weights.h5') # ذخیره وزن‌های هر کلاینت
    #print("**********",model.predict(x_test[:100].argmax(axis=1)))
    client_models.append(model)  # افزودن مدل به لیست
print("Training completed for all clients.")

Training model for client 1
Epoch 1/5
63/63 [==============================] - 0s 6ms/step - loss: 2.2745 - accuracy: 0.2150
Epoch 2/5
63/63 [==============================] - 0s 6ms/step - loss: 1.8639 - accuracy: 0.4620
Epoch 3/5
63/63 [==============================] - 0s 7ms/step - loss: 1.2572 - accuracy: 0.5810
Epoch 4/5
63/63 [==============================] - 0s 6ms/step - loss: 0.9718 - accuracy: 0.6795
Epoch 5/5
63/63 [==============================] - 0s 6ms/step - loss: 0.8391 - accuracy: 0.7285
Training model for client 2
Epoch 1/5
63/63 [==============================] - 0s 6ms/step - loss: 1.0026 - accuracy: 0.6870
Epoch 2/5
63/63 [==============================] - 0s 6ms/step - loss: 0.3553 - accuracy: 0.8895
Epoch 3/5
63/63 [==============================] - 0s 6ms/step - loss: 0.2412 - accuracy: 0.9295
Epoch 4/5
63/63 [==============================] - 0s 6ms/step - loss: 0.1583 - accuracy: 0.9480
Epoch 5/5
63/63 [==============================] - 0s 5ms/step - loss: 

### Class

In [9]:
def delta_class(models, x_test):
    num_models = len(models)
    argmax_preds = [model.predict(x_test).argmax(axis=1) for model in models]   # پیش‌بینی‌ها برای هر مدل
    diffs_matrix = np.zeros((num_models, num_models), dtype=int)    # ماتریس مربعی برای ذخیره تفاوت‌ها
    for i in range(num_models):       # پر کردن ماتریس با تفاوت پیش‌بینی‌ها
        for j in range(i+1, num_models):
            diffs = np.sum(argmax_preds[i] != argmax_preds[j])  # تعداد تفاوت‌های پیش‌بینی بین مدل i و مدل j
            diffs_matrix[i, j] = diffs
            diffs_matrix[j, i] = diffs  # ماتریس متقارن است
    return diffs_matrix

# اندازه‌گیری زمان
start_time = time.time()
diff_class = delta_class(client_models, x_test)
print("Difference Matrix for Delta Class:")
print(diff_class)

# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Difference Matrix for Delta Class:
[[   0 3056 2978 2821 2862 2966 3028 2894 2956 3121 3067 3051 2961 3093
  3032 1792 2882 2952 3166 1131 3002 2993 3033 3041 3054 2886 2937 2919
  2992 2877]
 [3056    0  820  838  809  848  794  830  814  909  795  764  737 1007
   793 3390  786  864  980 3136  915  955  831  838  789  816  851  899
   718 1038]
 [2978  820    0  747  767  808  743  795  748  832  881  764  745  992
   790 3365  687  768  856 3071  905  916  864  817  715  791  766  867
   752  903]
 [2821  838  747    0  797  837  851  786  798  852  885  823  720 1006
   841 3195  748  770 1009 2887  895  884  822  806  807  751  765  836
   744  960]
 [2862  809  767  797    0  843  827  822  761  943  886  843  682 1058
   855 3224  752  856 1032 2911  988  933  835  832  788  831  752  876
   767  953]
 [2966  848  808  837  843    0  802  839  874  984  902  838  841  949
   767 3385  869  827 1006 3067  866  929  907  832  784  876  912  878
   781 1027]
 [3028  794  743  851  

### Score

In [10]:
def delta_score(models, x_test, threshold):
    num_models = len(models)
    diffs_matrix = np.zeros((num_models, num_models))

    # پیش‌بینی خروجی‌های هر مدل
    predictions = [model.predict(x_test) for model in models]
    
    # محاسبه‌ی argmax برای هر مدل
    argmax_preds = [np.argmax(pred, axis=1) for pred in predictions]

    # مقایسه مدل‌ها دو به دو
    for i in range(num_models):
        for j in range(i + 1, num_models):
            diff_count = 0  # شمارنده اختلافات

            # مقایسه‌ی argmax ها
            argmax_diff = argmax_preds[i] != argmax_preds[j]
            diff_count += np.sum(argmax_diff)  # اضافه کردن اختلاف در argmax
            
            # مقایسه‌ی احتمالات برای پیش‌بینی‌های با argmax یکسان
            same_argmax_indices = np.where(argmax_preds[i] == argmax_preds[j])[0]
            
            for idx in same_argmax_indices:
                # مقایسه‌ی احتمالات در همان کلاس‌های argmax
                prob_i = round(predictions[i][idx][argmax_preds[i][idx]], 2)
                prob_j = round(predictions[j][idx][argmax_preds[j][idx]], 2)
                
                # محاسبه اختلاف بین پیش‌بینی‌ها
                prob_diff = abs(prob_i - prob_j)
                
                # بررسی اختلاف با آستانه threshold
                if prob_diff >= threshold:
                    diff_count += 1  # شمارش اختلاف در پیش‌بینی‌ها

            # ذخیره اختلافات در هر دو موقعیت (i, j) و (j, i) برای متقارن بودن
            diffs_matrix[i, j] = diff_count
            diffs_matrix[j, i] = diff_count

    return diffs_matrix

# اجرای تابع با تعیین آستانه threshold
diff_score = delta_score(client_models, x_test, threshold=0.1)
print("Distance Matrix for Delta Score:")
print(diff_score)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_score: {execution_time:.4f} seconds")


Distance Matrix for Delta Score:
[[   0. 3732. 3795. 3600. 3659. 3802. 3886. 3744. 3657. 3883. 3923. 3782.
  3712. 3989. 3779. 1890. 3619. 3759. 4009. 1232. 3927. 3908. 3902. 3881.
  3816. 3751. 3702. 3633. 3692. 3665.]
 [3732.    0. 2140. 2249. 2168. 2141. 2148. 2314. 2114. 2222. 2056. 2052.
  2052. 2473. 2083. 4056. 2028. 2209. 2421. 3819. 2369. 2415. 2215. 2251.
  2091. 2201. 2220. 2173. 1866. 2408.]
 [3795. 2140.    0. 2187. 2177. 2149. 2175. 2315. 2117. 2300. 2285. 2195.
  2122. 2497. 2087. 4166. 1953. 2193. 2288. 3926. 2418. 2478. 2399. 2323.
  2117. 2253. 2170. 2255. 2027. 2289.]
 [3600. 2249. 2187.    0. 2194. 2298. 2347. 2239. 2217. 2253. 2318. 2239.
  2087. 2502. 2270. 3902. 2138. 2142. 2664. 3654. 2467. 2358. 2196. 2265.
  2267. 2155. 2138. 2224. 2075. 2415.]
 [3659. 2168. 2177. 2194.    0. 2229. 2303. 2290. 2178. 2410. 2287. 2248.
  2061. 2580. 2294. 3977. 2059. 2261. 2649. 3715. 2517. 2447. 2242. 2265.
  2176. 2221. 2055. 2229. 2061. 2329.]
 [3802. 2141. 2149. 2298. 2229. 

### KS

In [11]:
def ks(models, x_test):
    num_samples = x_test.shape[0]
    num_models = len(models)

    # مرحله 1: پیش‌بینی کلاس‌ها برای داده‌های تست توسط هر مدل
    argmax_preds = np.array([model.predict(x_test).argmax(axis=1) for model in client_models])

    # مرحله 2: یافتن کلاس پرتکرار برای هر نمونه
    most_frequent_classes = [Counter(argmax_preds[:, i]).most_common(1)[0][0] for i in range(num_samples)]

    # مرحله 3: استخراج احتمال کلاس پرتکرار از تمامی مدل‌ها برای هر نمونه
    all_probabilities = np.zeros((num_samples, num_clients))
    for i, model in enumerate(client_models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            most_frequent_class = most_frequent_classes[j]
            all_probabilities[j, i] = predictions[j, most_frequent_class]  # احتمال کلاس پرتکرار
    # مرحله 4: انجام تست KS بین مدل‌های کلاینت و ایجاد ماتریس KS
    ks_distance_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            # تست KS برای مقایسه احتمالات دو مدل کلاینت
            ks_statistic, _ = ks_2samp(all_probabilities[:, i], all_probabilities[:, j])
            ks_distance_matrix[i, j] = ks_statistic
            ks_distance_matrix[j, i] = ks_statistic  # ماتریس متقارن

    return ks_distance_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_ks = ks(client_models, x_test)
print("Distance Matrix for KS:")
print(matrix_ks)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for KS:
[[0.     0.6837 0.6857 0.6735 0.6865 0.6806 0.6841 0.6813 0.6829 0.6795
  0.6865 0.682  0.6831 0.6828 0.6823 0.0416 0.6811 0.6823 0.6847 0.0214
  0.6765 0.6837 0.6851 0.683  0.6858 0.6836 0.6867 0.6757 0.6827 0.676 ]
 [0.6837 0.     0.0356 0.0568 0.0485 0.0364 0.0573 0.0455 0.0825 0.024
  0.0691 0.0226 0.0342 0.0574 0.0192 0.6422 0.043  0.0556 0.0789 0.6753
  0.0497 0.0486 0.0419 0.0287 0.0291 0.0632 0.0472 0.0562 0.0571 0.0998]
 [0.6857 0.0356 0.     0.0571 0.0165 0.0143 0.0266 0.0188 0.0919 0.0308
  0.0448 0.0188 0.0396 0.0349 0.0221 0.6445 0.0505 0.0244 0.0527 0.6765
  0.0296 0.0274 0.0189 0.0268 0.0148 0.0325 0.0573 0.0667 0.07   0.1071]
 [0.6735 0.0568 0.0571 0.     0.0694 0.0465 0.058  0.0477 0.0595 0.0438
  0.0836 0.0451 0.0338 0.0493 0.0551 0.632  0.0268 0.061  0.0772 0.6655
  0.0395 0.044  0.0515 0.0353 0.0591 0.0537 0.0515 0.043  0.0459 0.0707]
 [0.6865 0.0485 0.0165 0.0694 0.     0.0257 0.0203 0.027  0.1045 0.0441
  0.0316 0.0308 0.053  0.0294 0.0306 

### Chi2

In [12]:
def chi_square(models, x_test):
    num_models = len(models)
    num_samples = x_test.shape[0] # تعداد نمونه‌ها  
    n_classes = 10  # تعداد کلاس ها

    # ساخت ماتریس متقارن برای نتایج Chi-Square
    chi2_matrix = np.zeros((num_models, num_models))
    all_probabilities = np.zeros((num_samples, num_models))

    for i, model in enumerate(models):
        predictions = model.predict(x_test)  # احتمالات کلاس‌های مختلف برای هر نمونه
        for j, sample in enumerate(x_test):
            all_probabilities[j, i] = predictions[j].argmax()

    # مرحله: ایجاد جدول‌های متقاطع و انجام تست Chi-Square
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # ساخت جدول متقاطع
            contingency_table = pd.crosstab(all_probabilities[:, i], all_probabilities[:, j])
            chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
            chi2_matrix[i, j] = chi2_stat
            chi2_matrix[j, i] = chi2_stat  # ماتریس متقارن

    return chi2_matrix

# اجرای تابع با تعیین تعداد ارقام اعشار
matrix_chi2 = chi_square(client_models, x_test)
print("Distance Matrix for Chi square:")
print(matrix_chi2)

# start_time = time.time()
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"\nExecution time for delta_class: {execution_time:.4f} seconds")

Distance Matrix for Chi square:
[[    0.         44204.05264391 45219.64430531 46704.60303096
  45165.22662329 45758.02517846 45346.93394299 45732.71581961
  44652.72888619 44189.32797093 44606.01625262 44963.68993944
  44856.1157383  43381.58023074 45227.04395038 52485.17251841
  46014.08751545 46073.99255923 43506.97094915 61638.7852426
  45245.70402613 44999.76562982 44858.00216026 45194.63172083
  44755.93310349 46013.81524538 45200.96727567 45461.43901741
  45244.36771994 44747.52845805]
 [44204.05264391     0.         74579.26941519 74248.82223184
  74549.40471335 74038.78813315 74905.71110643 74252.05515549
  74512.02711113 73274.66281022 74913.51974548 75494.64662477
  75921.49507832 71673.82163319 75109.75332241 40716.97985303
  75089.72913101 73775.15821024 72027.28229034 43669.85835685
  72899.34957354 72351.74899499 74462.63335868 74151.97601477
  75055.12230505 74533.82028145 73965.70549347 73439.58981652
  76158.03674885 70680.30694156]
 [45219.64430531 74579.26941519    

### The problematic client

In [ ]:
def meancal(matrix):
    temp = 0
    x = matrix.shape[0]    
    arrmean = []
    
    for i in range(0,x):
        #print(matrix[i].mean())
        temp = (matrix[i].mean())    
        arrmean.append(temp)
    return arrmean

def iqrfunc(nparray):
    data = np.array(nparray)
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)
    iqr = q3 -q1
    lower_bound = q1-(iqr*1.5)
    upper_bound = q3+(iqr*1.5)
    outliers = np.where((data<lower_bound) | (data>upper_bound))[0]
    return outliers
    
diff_class = delta_class(client_models, x_test)
diff_score = delta_score(client_models, x_test,0.1)
diff_ks = ks(client_models, x_test)
diff_chi2 = chi_square(client_models, x_test)

temp_c = meancal(diff_class)
temp_s = meancal(diff_score)
temp_ks = meancal(diff_ks)
temp_chi2 = meancal(diff_chi2)

print(f"Problemmatic Clients for Delta class {iqrfunc(temp_c)}:")                       
print(f"Problemmatic Clients for Delta Score {iqrfunc(temp_s)}:")                                            
print(f"Problemmatic Clients for KS {iqrfunc(temp_ks)}:")                       
print(f"Problemmatic Clients for X2 {iqrfunc(temp_chi2)}:") 